In [3]:
%config Completer.use_jedi = False

In [4]:
import dataiku
import inference
from elasticsearch import Elasticsearch

es = Elasticsearch()

def es_match_query(payload, es, k=3):
    idx_name = 'idx_zalando'
    search_body = {
        "_source": {
            "excludes": ["zalando_nlu_vector"]
        },
        "highlight": {
            "fields": {
                "description": {}
            }
        },
        "query": {
            "match": {
                "description": {
                    "query": payload
                }
            }
        }
    }

    search_response = es.search(request_timeout=30, index=idx_name,
                                body=search_body)['hits']['hits'][:k]

    response = [{'image': x['_source']['image'], 'description': x['highlight']['description']} for x in search_response]

    return response

def generate_image_url(name):
    return f'http://localhost:11500/dip/api/managedfolder/preview-image?projectKey=FASHIONSEARCH&odbId=wQPWIBlM&itemPath={name}&contentType=image/jpeg'



In [22]:
def api_py_function(query, k):
    
    search = es_match_query(query, es, k)
    
    for i in range(len(search)):
        search[i]['presigned_url'] = generate_image_url([search[i]['image']][0])
        search[i]['description'] = " ".join(search[i]['description'])
        search[i]['description'] = search[i]['description'].replace("<em>",'<em style="background-color:#f18973;">')

    return search

In [20]:


    #response = es.search(index='idx_zalando', **search_query)
    #results = [r['_source']['image'] for r in response['hits']['hits']]
    #image_urls = [f'http://localhost:11500/dip/api/managedfolder/preview-image?projectKey=FASHIONSEARCH&odbId=wQPWIBlM&itemPath={i}&contentType=image/jpeg' for i in results]
      
    #return {'images': image_urls}
    #return response

In [21]:
api_py_function('blue dress', 3)

/Users/davidcarlson/Library/dss/design/code-envs/python/search/lib/python3.6/site-packages/ipykernel_launcher.py:22: DeprecationWarning: The 'body' parameter is deprecated for the 'search' API and will be removed in a future version. Instead use API parameters directly. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information


[{'image': 'BA221C00I-K11@12.jpg',
  'description': 'Short white <em style="background-color:#f18973;">dress</em> with square <em style="background-color:#f18973;">blue</em> pattern and round neckline and short sleeves. <em style="background-color:#f18973;">Blue</em> belt and slightly waisted.,Knee-length fabric <em style="background-color:#f18973;">dress</em> with white-<em style="background-color:#f18973;">blue</em> checked pattern, girdles, short sleeves and round neckline.,white <em style="background-color:#f18973;">blue</em> <em style="background-color:#f18973;">dress</em> in a tarnished shape, it has a <em style="background-color:#f18973;">blue</em> very thin belt and a round neckline,A <em style="background-color:#f18973;">blue</em> and white checked mini <em style="background-color:#f18973;">dress</em>. It has a slim <em style="background-color:#f18973;">blue</em> belt with a buckle.,Mid-length, dark <em style="background-color:#f18973;">blue</em> and white plaid <em style="bac

In [0]:
inference.